In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
df_silver_customers = spark.read.table(f'{catalog}.{schema}.silver_customers')
df_silver_customers.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.dim_customers")

In [0]:
df_silver_products = spark.read.table(f'{catalog}.{schema}.silver_products')
df_silver_products.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.dim_product")


In [0]:
df_silver_seller = spark.read.table(f'{catalog}.{schema}.silver_sellers')
df_silver_seller.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.dim_seller")


In [0]:
df_silver_orders = spark.read.table(f'{catalog}.{schema}.silver_orders')
df_dim_date = df_silver_orders.select("order_purchase_timestamp").withColumn("order_purchase_timestamp",col("order_purchase_timestamp").cast("date")).distinct()
# --96476  612
# df_dim_date.count()
df_dim_date.withColumn("day",dayofmonth(col("order_purchase_timestamp"))) \
    .withColumn("month",month(col("order_purchase_timestamp"))) \
    .withColumn("year",year(col("order_purchase_timestamp"))) \
    .withColumn("year",weekofyear(col("order_purchase_timestamp"))) \
    .withColumn("quarter",quarter(col("order_purchase_timestamp"))) \
    .withColumnRenamed("order_purchase_timestamp","order_pur_dt")

df_dim_date.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.dim_date")


In [0]:
df_silver_orders = spark.read.table(f'{catalog}.{schema}.silver_orders')
df_silver_customers = spark.read.table(f'{catalog}.{schema}.silver_customers')

df_forders = df_silver_orders.join(df_silver_customers,on = 'customer_id',how = 'inner').select("order_id","customer_id","order_status","order_purchase_timestamp","delivery_days","delay_flag")
df_forders.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.fact_orders")

In [0]:
df_s_orders = spark.read.table(f'{catalog}.{schema}.silver_orders')
df_s_oitems = spark.read.table(f'{catalog}.{schema}.silver_order_items')
df_s_payments = spark.read.table(f'{catalog}.{schema}.silver_payments')

df_comb_data = df_s_oitems.join(df_s_orders,on = 'order_id',how = 'inner')
df_comb_data1 = df_comb_data.join(df_s_payments,on = 'order_id',how = 'inner')

df_fsale = df_comb_data1.select("order_id","product_id","seller_id","customer_id","payment_type","price","freight_value","item_total_value","order_purchase_timestamp")

df_rev = df_fsale.groupBy("customer_id").agg(sum("item_total_value").alias("revenue"))

df_fact_sales = df_fsale.join(df_rev,on = "customer_id",how = "inner")
df_fact_sales.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.fact_sales")